In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = True

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [4]:
simp = True # let's leave this true for now because of the way we calculate the piecewise discrete "derivative" below. Seems to result in vastly simpler equations.

### TODO either show convexity for everything or verify optimality otherwise (i.e. by simply plugging into offchain-tests)

## Equalities

In [5]:
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if delta_b_negative and asset == 'b':
    f = -floor(f)
  else:
    f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

b_b + v_b - floor(s_b/w_b)

In [6]:
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = floor(a * ((1 + 1/j) ** e))
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

floor(a_b*((j_b + 1)/j_b)**e_b)

In [7]:
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
  f = deltaBySpot[asset].subs(s, spotByExp[asset])
  if simp:
    f = simplify(f)
  return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)

In [8]:
deltaByExp['s']

-b_s - v_s + ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [9]:
if delta_b_negative:
  eff = Delta_s / -Delta_b
else:
  eff = Delta_s / Delta_b
eff

-Delta_s/Delta_b

In [10]:
effByExps = eff.subs(Delta_s, deltaByExp['s']).subs(Delta_b, deltaByExp['b'])
if simp:
  effByExps = simplify(effByExps)
effByExps

(b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))/(b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))

## Inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

### value cannot decrease

In [11]:
a0Buying = spotByExp['s'] * deltaByExp['b'] # < 0 if delta_b_negative, > 0 otherwise
if delta_b_negative:
    a0Buying = -a0Buying # > 0
if simp:
  a0Buying = simplify(a0Buying)
a0Buying

(-b_b - v_b + floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s)

In [12]:
a0Selling = spotByExp['b'] * deltaByExp['s'] # > 0
if simp:
  a0Selling = simplify(a0Selling)
a0Selling

(-b_s - v_s + ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b)

In [13]:
a0Bound = a0Buying - a0Selling # <=! 0
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

-(b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) + (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b)

### exponents cannot result in spot prices that are better than the amm-prices

In [14]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [15]:
expBound['s']

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

### spot prices cannot exceed maxInteger

In [16]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

maxSpotBound_b = spotByExp['b'] - I_max # <=! 0
if simp:
  maxSpotBound_b = simplify(maxSpotBound_b)
maxSpotBound_b

-I_max + floor(a_b*(j_b + 1)**e_b/j_b**e_b)

In [17]:
maxSpotBound_s = spotByExp['s'] - I_max # <=! 0
if simp:
  maxSpotBound_s = simplify(maxSpotBound_s)
maxSpotBound_s

-I_max + floor(a_s*(j_s + 1)**e_s/j_s**e_s)

### need to swap at least minimum amounts

In [18]:
if delta_b_negative:
  minAmntBound_b = min_b + deltaByExp['b'] # <=! 0
else:
  minAmntBound_b = min_b - deltaByExp['b'] # <=! 0
if simp:
  minAmntBound_b = simplify(minAmntBound_b)
minAmntBound_b

b_b + min_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)

In [19]:
minAmntBound_s = min_s - deltaByExp['s'] # <=! 0
if simp:
  minAmntBound_s = simplify(minAmntBound_s)
minAmntBound_s

b_s + min_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)

### need to swap at most maximum amounts

In [20]:
if delta_b_negative:
  maxAmntBound_b = -deltaByExp['b'] - b_b # <=! 0
else:
  maxAmntBound_b = deltaByExp['b'] - b_b # <=! 0
if simp:
  maxAmntBound_b = simplify(maxAmntBound_b)
maxAmntBound_b

-2*b_b - v_b + floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)

In [21]:
maxAmntBound_s = deltaByExp['s'] - max_s # <=! 0
maxAmntBound_s

-b_s - max_s - v_s + ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)

### total number of multiplications of both exponentiations cannot exceed expLimit

In [22]:
# expLimit, aka maximum total number of multiplications for both exponents
m_max = symbols('m_max', positive=True, integer=True)

def numMultsLower_(asset, s,v, b, w, j, e, Delta, a, min):
  return floor(log(e, 2))

# the best number of multiplications we can get by increasing e
def bestMultsAhead_(asset, s,v, b, w, j, e, Delta, a, min):
  return ceiling(log(e, 2))

bestMultsAhead = {asset: bestMultsAhead_(asset, *all_symbols[asset]) for asset in assets}
bestMultsAhead['b']

ceiling(log(e_b)/log(2))

In [23]:
# optimistic estimate. This relies on the vague intuition that exponents will be minimal from the other constraints, so best we can do is increase them if this one is violated
optimisticMultsBound = bestMultsAhead['b'] + bestMultsAhead['s'] - m_max # <=! 0
if simp:
  optimisticMultsBound = simplify(optimisticMultsBound)
optimisticMultsBound

-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))

## Lagrangian

In [24]:
# Decision variables
xs = [e_b, e_s]

# Equality-Constraints
hs = [] 

# Inequality-Constraints
gs = [
    expBound['b'],
    expBound['s'],
    maxSpotBound_b,
    maxSpotBound_s,
    minAmntBound_b,
    minAmntBound_s,
    maxAmntBound_b,
    maxAmntBound_s,
    a0Bound,
    optimisticMultsBound,
] 

# Lagrange multipliers for equality- and inequality-constraints
mus = symbols('mu_1:{}'.format(len(hs)+1), nonnegative=True)
lambdas = symbols('lambda_1:{}'.format(len(gs)+1), nonnegative=True)

# Objective function
f = effByExps

# Lagrangian
L = f + sum([mu * h for mu, h in zip(mus, hs)]) + sum([lambda_ * g for lambda_, g in zip(lambdas, gs)])
# if simp: # not using this anyways
#     L = simplify(L)
L

lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))) + lambda_10*(-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))) + lambda_2*(-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))) + lambda_3*(-I_max + floor(a_b*(j_b + 1)**e_b/j_b**e_b)) + lambda_4*(-I_max + floor(a_s*(j_s + 1)**e_s/j_s**e_s)) + lambda_5*(b_b + min_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)) + lambda_6*(b_s + min_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)) + lambda_7*(-2*b_b - v_b + floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)) + lambda_8*(-b_s - max_s - v_s + ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)) + lambda_9*(-(b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) + (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b)) + (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))/(b_b + v_b - floor(floor(a_b*(j_b + 

In [25]:
# Equations from stationarity

def discrete_diff(f, n):
  diff = f.subs(n, n+1) - f
  if simp:
    diff = simplify(diff)
  return diff

equations = []
for x in xs:
  dL_dx = discrete_diff(f, x)\
    + sum([discrete_diff(mu * h, x) for mu, h in zip(mus, hs)])\
    + sum([discrete_diff(lambda_ * g, x) for lambda_, g in zip(lambdas, gs)]) # probably slightly better to do it this way instead of simply using L
  equation = Eq(dL_dx, 0)
  # if simp: # results in error from the bestMultsAhead-constraint
  #   equation = simplify(equation)
  display(equation)
  equations.append(equation)

Eq(lambda_1 + lambda_10*(-ceiling(log(e_b)/log(2)) + ceiling(log(e_b + 1)/log(2))) + lambda_3*(-floor(a_b*(j_b + 1)**e_b/j_b**e_b) + floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1))) + lambda_5*(floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b) - floor(floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1))/w_b)) + lambda_7*(-floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b) + floor(floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1))/w_b)) + lambda_9*((b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) - (b_b + v_b - floor(floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1))/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) - (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b) + (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1))) + ((b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*(b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)) + (b_b + v_b - floor(fl

Eq(lambda_10*(-ceiling(log(e_s)/log(2)) + ceiling(log(e_s + 1)/log(2))) - lambda_2 + lambda_4*(-floor(a_s*(j_s + 1)**e_s/j_s**e_s) + floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1))) + lambda_6*(ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s) - ceiling(floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1))/w_s)) + lambda_8*(-ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s) + ceiling(floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1))/w_s)) + lambda_9*((b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) - (b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1)) - (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b) + (b_s + v_s - ceiling(floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1))/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b)) + (ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s) - ceiling(floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1))/w_s))/(b_b + v_b - floor(floor(

In [26]:
# Primal feasibility conditions for equality-constraints
for h in hs:
    equation = Eq(h, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

In [27]:
# Complementary slackness conditions
for lambda_, g in zip(lambdas, gs):
    equation = Eq(lambda_ * g, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))), 0)

Eq(lambda_2*(e_s - log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))), 0)

Eq(lambda_3*(-I_max + floor(a_b*(j_b + 1)**e_b/j_b**e_b)), 0)

Eq(lambda_4*(-I_max + floor(a_s*(j_s + 1)**e_s/j_s**e_s)), 0)

Eq(lambda_5*(b_b + min_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)), 0)

Eq(lambda_6*(b_s + min_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)), 0)

Eq(lambda_7*(-2*b_b - v_b + floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b)), 0)

Eq(lambda_8*(-b_s - max_s - v_s + ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s)), 0)

Eq(lambda_9*((b_b + v_b - floor(floor(a_b*(j_b + 1)**e_b/j_b**e_b)/w_b))*floor(a_s*(j_s + 1)**e_s/j_s**e_s) - (b_s + v_s - ceiling(floor(a_s*(j_s + 1)**e_s/j_s**e_s)/w_s))*floor(a_b*(j_b + 1)**e_b/j_b**e_b)), 0)

Eq(lambda_10*(-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))), 0)

In [28]:
def verify_inequality_constraints(solution, vars, gs):
    subs_dict = dict(zip(vars, solution))
    for g in gs:
        if Gt(g.subs(subs_dict), 0) == True:
            return False
    return True

In [29]:
# Solve
vars = xs + list(mus) + list(lambdas)
solutions = nonlinsolve(equations, vars)
len(solutions)

1

In [30]:
solutions_ = [solution for solution in solutions if verify_inequality_constraints(solution, vars, gs)]
len(solutions_)

1